# F1 Podium Predictor (Kaggle CSVs) — Baseline Notebook

This notebook builds a **baseline classifier** to predict whether a driver finishes on the podium (top 3) using historical F1 data from a Kaggle dataset (Ergast mirror).

## What you'll do
1. (Optional) Download a Kaggle dataset of Formula 1 results directly from this notebook (requires your Kaggle API token).
2. Load `races.csv`, `results.csv`, and `qualifying.csv`.
3. Normalize column names and merge into a modeling table.
4. Create a **podium** label, and train a **Logistic Regression** baseline using only `grid`.
5. Evaluate and visualize simple insights.

> **Tip**: If you already downloaded CSVs manually, place them under `data/raw/` and skip to **Load CSVs**.

## 0. Environment setup
Run this cell once per environment. If you're in a managed environment without internet, comment out the installs and just ensure the packages are available.

In [ ]:
# If needed, install dependencies (uncomment as necessary)
# %pip install -q pandas scikit-learn matplotlib kaggle python-dotenv

## 1. (Optional) Kaggle download from notebook
You need your Kaggle API token file at `~/.kaggle/kaggle.json`.

### Steps (once):
1. Go to Kaggle → Account → **Create New API Token**; this downloads `kaggle.json`.
2. Move it to `~/.kaggle/kaggle.json` and set permissions `chmod 600 ~/.kaggle/kaggle.json`.
3. Set your dataset slug below and run the cell to download to `data/raw/`.

In [1]:
import os, pathlib, sys, subprocess, shutil

RAW_DIR = pathlib.Path("data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)

# <-- Update this to the dataset you chose on Kaggle (examples in markdown below) -->
KAGGLE_DATASET = "rohanrao/formula-1-world-championship-1950-2020"  # e.g., "rohanrao/formula-1-world-championship-1950-2020" or similar

token_path = pathlib.Path.home()/".kaggle/kaggle.json"
if KAGGLE_DATASET:
    if token_path.exists():
        print("Found Kaggle token at", token_path)
        # Download using Kaggle CLI via subprocess (works in local envs)
        try:
            subprocess.run(
                ["kaggle", "datasets", "download", "-d", KAGGLE_DATASET, "-p", str(RAW_DIR), "--unzip"],
                check=True
            )
            print("Downloaded to", RAW_DIR)
        except Exception as e:
            print("Failed to download via kaggle CLI:", e)
            print("Tip: Ensure Kaggle CLI is installed: `pip install kaggle` and that kaggle is on PATH.")
    else:
        print("Kaggle token not found at ~/.kaggle/kaggle.json. Skipping download. "
              "Place CSVs in data/raw/ and continue.")

Found Kaggle token at /Users/tsuzumi.sato/.kaggle/kaggle.json
Dataset URL: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020
License(s): CC0-1.0

Downloaded to data/raw


100%|██████████| 6.28M/6.28M [00:00<00:00, 3.52GB/s]


In [2]:
!which python
!pip show kaggle

/Users/tsuzumi.sato/.pyenv/shims/python


**Popular Kaggle F1 datasets (examples):**
- `rohanrao/formula-1-world-championship-1950-2020`
- `muhammadjunaidshahbaz/f1-races-1950-2024`  (if available)
- Any Ergast mirrors containing `races.csv`, `results.csv`, `qualifying.csv`

Your files should end up like:
```
data/raw/races.csv
data/raw/results.csv
data/raw/qualifying.csv
```
Column names vary slightly; this notebook normalizes them.

## 2. Imports & config

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Paths
RAW = Path("data/raw")
PROCESSED = Path("data/processed")
PROCESSED.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 100)

## 3. Load CSVs
Load the three main files. If your dataset has different names, adjust the paths below.

In [3]:
races_path = RAW / "races.csv"
results_path = RAW / "results.csv"
qualifying_path = RAW / "qualifying.csv"

assert races_path.exists(), f"Missing {races_path}"
assert results_path.exists(), f"Missing {results_path}"
assert qualifying_path.exists(), f"Missing {qualifying_path}"

races = pd.read_csv(races_path)
results = pd.read_csv(results_path)
qualifying = pd.read_csv(qualifying_path)

races.head(3), results.head(3), qualifying.head(3)

(   raceId  year  round  circuitId                   name        date  \
 0       1  2009      1          1  Australian Grand Prix  2009-03-29   
 1       2  2009      2          2   Malaysian Grand Prix  2009-04-05   
 2       3  2009      3         17     Chinese Grand Prix  2009-04-19   
 
        time                                                url fp1_date  \
 0  06:00:00  http://en.wikipedia.org/wiki/2009_Australian_G...       \N   
 1  09:00:00  http://en.wikipedia.org/wiki/2009_Malaysian_Gr...       \N   
 2  07:00:00  http://en.wikipedia.org/wiki/2009_Chinese_Gran...       \N   
 
   fp1_time fp2_date fp2_time fp3_date fp3_time quali_date quali_time  \
 0       \N       \N       \N       \N       \N         \N         \N   
 1       \N       \N       \N       \N       \N         \N         \N   
 2       \N       \N       \N       \N       \N         \N         \N   
 
   sprint_date sprint_time  
 0          \N          \N  
 1          \N          \N  
 2          \N     

In [6]:
print("Results columns: ", results.columns.tolist())
print("Races columns: ", races.columns.tolist())

Results columns:  ['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId']
Races columns:  ['raceId', 'season', 'round', 'circuitId', 'name', 'date', 'time', 'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time']


## 4. Normalize column names
Different mirrors use slightly different cases. We coerce common keys.

In [4]:
def _normalize_id_cols(df):
    lc = {c.lower(): c for c in df.columns}
    # Map lower-case names to canonical
    rename_map = {}
    if 'year' in lc and 'season' not in df.columns:
        rename_map[lc['year']] = 'season'
    for k in ['driverid','constructorid','raceid','circuitid']:
        if k in lc and k[:-2]+'Id' not in df.columns:
            # e.g., 'driverid' -> 'driverId'
            rename_map[lc[k]] = k[:-2] + 'Id'
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

races = _normalize_id_cols(races)
results = _normalize_id_cols(results)
qualifying = _normalize_id_cols(qualifying)

sorted((races.columns)), sorted((results.columns)), sorted((qualifying.columns))

(['circuitId',
  'date',
  'fp1_date',
  'fp1_time',
  'fp2_date',
  'fp2_time',
  'fp3_date',
  'fp3_time',
  'name',
  'quali_date',
  'quali_time',
  'raceId',
  'round',
  'season',
  'sprint_date',
  'sprint_time',
  'time',
  'url'],
 ['constructorId',
  'driverId',
  'fastestLap',
  'fastestLapSpeed',
  'fastestLapTime',
  'grid',
  'laps',
  'milliseconds',
  'number',
  'points',
  'position',
  'positionOrder',
  'positionText',
  'raceId',
  'rank',
  'resultId',
  'statusId',
  'time'],
 ['constructorId',
  'driverId',
  'number',
  'position',
  'q1',
  'q2',
  'q3',
  'qualifyId',
  'raceId'])

## 5. Select essentials & merge
We build a modeling table keyed by `raceId` + `driverId`. We join qualifying to results, and also attach race metadata for labels.

In [9]:
# Select columns
results_cols = ['raceId','driverId','constructorId','grid','position','points']
if 'statusId' in results.columns:
    results_cols.insert(5, 'statusId')

r = results[results_cols].copy()

q = qualifying[['raceId','driverId','position']].copy()
q = q.rename(columns={'position':'grid_quali'})

# Safe numeric conversion
def to_int(x):
    try:
        return int(x)
    except Exception:
        return None

r['grid'] = r['grid'].apply(to_int)
r['position'] = r['position'].apply(to_int)

df = r.merge(q, on=['raceId','driverId'], how='left')

if 'year' in races.columns:
    races = races.rename(columns={'year': 'season'})

race_meta = races[['raceId','season','round','name','date','circuitId']].copy()
df = df.merge(race_meta, on=['raceId'], how='left')

print(df.shape)
df.head(5)

(26759, 13)


,raceId,driverId,constructorId,grid,position,statusId,points,grid_quali,season,round,name,date,circuitId
0,18,1,1,1,1.0,1,10.0,1.0,2008,1,Australian Grand Prix,2008-03-16,1
1,18,2,2,5,2.0,1,8.0,5.0,2008,1,Australian Grand Prix,2008-03-16,1
2,18,3,3,7,3.0,1,6.0,7.0,2008,1,Australian Grand Prix,2008-03-16,1
3,18,4,4,11,4.0,1,5.0,12.0,2008,1,Australian Grand Prix,2008-03-16,1
4,18,5,1,3,5.0,1,4.0,3.0,2008,1,Australian Grand Prix,2008-03-16,1


## 6. Define target & simple filtering
For a baseline, we'll:
- Keep only cars classified as **finished** (simplifies the first pass)
- Create `podium = 1 if position ≤ 3 else 0`

In [10]:
# finish_like = df['status'].astype(str).str.contains('Finished|\\+1 Lap|\\+2 Laps|\\+3 Laps', case=False, na=False)
# df_baseline = df[finish_like].copy()
# df_baseline['podium'] = df_baseline['position'].apply(lambda x: 1 if x is not None and x <= 3 else 0)

# print(df_baseline.shape)
# df_baseline[['season','round','name','driverId','grid','position','podium']].head(10)

df['position'] = pd.to_numeric(df['position'], errors='coerce')

if 'status' not in df.columns and 'statusId' in df.columns:
    status_path = Path("data/raw/status.csv")
    if status_path.exists():
        status_map

KeyError: 'status'

## 7. Save processed table

In [ ]:
out_path = PROCESSED / "baseline_dataset.csv"
df_baseline.to_csv(out_path, index=False)
out_path, len(df_baseline)

## 8. Baseline model — Logistic Regression (grid → podium)
We split **by season** using `GroupShuffleSplit` so the test set simulates unseen seasons.

In [ ]:
model_df = df_baseline.dropna(subset=['grid']).copy()
X = model_df[['grid']].astype(float)
y = model_df['podium'].astype(int)
groups = model_df['season']

gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))

## 9. Quick visual checks
We look at **podium rate by grid** (test split only) to understand the monotonic trend.

In [ ]:
test = model_df.iloc[test_idx].copy()
test['pred'] = y_pred

# Podium rate by grid (actual)
rate = test.groupby('grid')['podium'].mean()

plt.figure()
rate.plot(kind='bar')
plt.title('Actual Podium Rate by Grid (Test Split)')
plt.xlabel('Grid')
plt.ylabel('Podium Rate')
plt.tight_layout()
plt.show()

## 10. Next steps
- Add **driver/team form** features (rolling avg points last 3–5 races).
- Include **track type** (street vs permanent) and **qualifying gap to pole**.
- Try **XGBoost** or **Random Forest** for non-linear interactions.
- Calibrate probabilities; add reliability plot.
- Build a small Streamlit app using the saved model.